In [1]:
import ast
import json
import re

import nltk
import pandas as pd
import pymorphy2
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm

In [2]:
# %%capture
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pocan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pocan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pocan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import warnings

warnings.filterwarnings('ignore')

In [4]:
def pymorphy2_311_hotfix():
    from inspect import getfullargspec
    from pymorphy2.units.base import BaseAnalyzerUnit

    def _get_param_names_311(klass):
        if klass.__init__ is object.__init__:
            return []
        args = getfullargspec(klass.__init__).args
        return sorted(args[1:])

    setattr(BaseAnalyzerUnit, '_get_param_names', _get_param_names_311)


pymorphy2_311_hotfix()

In [5]:
articles = pd.read_csv('data/articles.csv')
comments = pd.read_csv('data/comments.csv')

In [6]:
comments['published_datetime'] = pd.to_datetime(comments['published_datetime'])

comments.dtypes

article_id                          int64
author                             object
published_datetime    datetime64[ns, UTC]
text                               object
votes                               int64
dtype: object

In [7]:
comments.sample(10)

,article_id,author,published_datetime,text,votes
6195,767274,nameless323,2023-10-21 16:12:51+00:00,а то вы меня совсем заминусуете с причиной\nЯ ...,0
11394,741040,Ahuromazdie,2023-06-10 09:02:52+00:00,Лучше просто уточнить в какой среде (хотя врод...,1
9322,756242,rob2akhiyarov,2023-08-27 19:31:58+00:00,"Мне не нравится то, как tailwind в своем основ...",0
304,798923,Budarin,2024-03-09 01:46:04+00:00,я правда не понял ваш вопрос про концепции\nсф...,0
14063,719352,nik_savchenko,2023-02-28 12:37:43+00:00,"Спасибо, думаю, этого достаточно)",0
5434,771438,bquadro,2023-11-17 12:31:01+00:00,Всем спасибо за уточнения про неточности в код...,0
12042,736598,kai3341,2023-05-22 00:16:47+00:00,для меня было важным критерием 500+ звёзд\nВы ...,2
2346,787606,aamonster,2024-01-20 13:08:43+00:00,Если ссылаться на математику – то дойдём до то...,0
17296,700964,Ilusha,2022-11-28 09:24:38+00:00,Ваша задача решается проще через vue2.7+. Вы м...,0
18471,687520,kulikofff,2022-09-21 21:45:05+00:00,Спасибо.,2


In [8]:
articles['published_datetime'] = pd.to_datetime(articles['published_datetime'])
articles['tags'] = articles['tags'].astype(str).apply(ast.literal_eval)

articles.dtypes

id                                  int64
author                             object
published_datetime    datetime64[ns, UTC]
title                              object
url                                object
complexity                         object
reading_time                        int64
views                               int64
tags                               object
votes                               int64
bookmarks                           int64
comments                            int64
dtype: object

In [9]:
articles.sample(10)

,id,author,published_datetime,title,url,complexity,reading_time,views,tags,votes,bookmarks,comments
986,686748,fazdendev,2022-09-06 14:41:05+00:00,Добавление масштабирования страницы при ширине...,https://habr.com/ru/articles/686748/,NaN,5,8200,"[Веб-разработка, CSS, HTML]",3,20,15
825,708850,Lavitard,2023-01-01 20:48:38+00:00,Удобный прогноз северного сияния: в цветах и г...,https://habr.com/ru/articles/708850/,NaN,10,15000,"[Веб-разработка, JavaScript, Интерфейсы]",20,20,6
174,769150,Green21,2024-01-11 08:57:39+00:00,Пишем 2D игру на JavaScript и Canvas. Часть 2....,https://habr.com/ru/articles/769150/,Простой,18,5500,"[Веб-разработка, JavaScript, Разработка игр, C...",11,60,12
226,779492,cherkalexander,2023-12-12 08:57:17+00:00,Как автоматизировать использование дизайн токе...,https://habr.com/ru/articles/779492/,Простой,8,3200,"[Веб-разработка, TypeScript, Дизайн]",5,42,4
730,720528,shasoftX,2023-03-06 05:15:39+00:00,Samoyed CMG — пользовательские генераторы кодо...,https://habr.com/ru/articles/720528/,Простой,7,642,"[Веб-разработка, PHP]",1,6,0
247,778316,VeXell,2023-12-04 12:33:09+00:00,Улучшаем возможности PM2 с помощью дополнитель...,https://habr.com/ru/articles/778316/,Простой,6,2300,"[Веб-разработка, JavaScript, IT-инфраструктура...",11,44,2
359,764888,xtremespb,2023-10-05 10:20:51+00:00,Heretic: full stack фреймворк на основе Marko.js,https://habr.com/ru/articles/764888/,Средний,7,3500,"[Веб-дизайн, Веб-разработка, JavaScript]",5,18,1
627,733302,mixkorshun,2023-05-10 14:23:56+00:00,7 ошибок глобального рефакторинга и как их изб...,https://habr.com/ru/articles/733302/,NaN,5,6300,"[Веб-разработка, Проектирование и рефакторинг]",11,49,5
269,772500,Konstantin_Loginovskikh,2023-11-22 11:56:03+00:00,Frontend Talks: как упростить создание контекс...,https://habr.com/ru/companies/cloud_ru/article...,NaN,7,2100,"[Блог компании Cloud.ru, Веб-разработка, Прогр...",3,12,3
591,737856,ph_piter,2023-05-31 14:16:33+00:00,Книга «JavaScript с нуля до профи»,https://habr.com/ru/companies/piter/articles/7...,NaN,14,17000,"[Блог компании Издательский дом «Питер», Веб-р...",17,111,5


In [10]:
print(
    f"Количество статей: {articles.shape[0]}",
    f"Количество комментариев: {comments.shape[0]}",
    '\n'
    f"Количество уникальных авторов постов: {articles['author'].nunique()}",
    f"Количество уникальных комментаторов: {comments['author'].nunique()}",
    '\n'
    f"Период времени сбора информации: {comments['published_datetime'].min().strftime('%Y-%m-%d')} по {comments['published_datetime'].max().strftime('%Y-%m-%d')}",
    sep='\n'
)


Количество статей: 999
Количество комментариев: 18750

Количество уникальных авторов постов: 571
Количество уникальных комментаторов: 5238

Период времени сбора информации: 2022-08-31 по 2024-03-16


In [11]:
def clean(text: str) -> str:
    text = re.sub(
        r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', ' ',
        text)
    text = re.sub(r'\b\d+\-\w+\b', ' ', text)
    text = re.sub(r'\\[nt]', ' ', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'_', ' ', text)
    text = re.sub(r'\b\d+\b', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


def tokenize(text):
    return word_tokenize(text, language="russian")


stop_words = set()
stop_words |= set(stopwords.words('russian'))  # default corpus
stop_words |= set(json.load(open('stopwords-ru.json', encoding='utf-8')))
stop_words |= set(chr(i) for i in range(ord('а'), ord('а') + 32))  # ru letters
stop_words |= set(chr(i) for i in range(ord('a'), ord('a') + 26))  # en letters
stop_words |= set(str(i) for i in range(10))  # digits
stop_words |= {'net', 'ru', 'com', 'org'}  # domens


def purge(tokens: list[str]) -> list[str]:
    return [t for t in tokens if t not in stop_words]


def lemmatizer():
    morphy_lemmatizer = pymorphy2.MorphAnalyzer().parse
    nltk_lemmatizer = WordNetLemmatizer().lemmatize

    def wrapper(tokens: list[str]) -> list[str]:
        return [morphy_lemmatizer(nltk_lemmatizer(token))[0].normal_form for token in tokens]

    return wrapper


In [12]:
comments['tokens'] = comments['text'].astype(str)

In [13]:
tqdm.pandas(desc='lower')
comments['tokens'] = comments['tokens'].progress_apply(str.lower)

lower:   0%|          | 0/18750 [00:00<?, ?it/s]

In [14]:
tqdm.pandas(desc='clean')
comments['tokens'] = comments['tokens'].progress_apply(clean)


clean:   0%|          | 0/18750 [00:00<?, ?it/s]

In [15]:
tqdm.pandas(desc='tokenize')
comments['tokens'] = comments['tokens'].progress_apply(tokenize)

tokenize:   0%|          | 0/18750 [00:00<?, ?it/s]

In [16]:
tqdm.pandas(desc='lemmatize')
comments['tokens'] = comments['tokens'].progress_apply(lemmatizer())

lemmatize:   0%|          | 0/18750 [00:00<?, ?it/s]

In [17]:
tqdm.pandas(desc='purge')
comments['purged_tokens'] = comments['tokens'].progress_apply(purge)

purge:   0%|          | 0/18750 [00:00<?, ?it/s]

In [18]:
comments.to_pickle('data/processed_comments.pkl')
articles.to_pickle('data/processed_articles.pkl')

In [31]:
text, tokens, purged_tokens = comments.sample(1)[['text', 'tokens', 'purged_tokens']].iloc[0]

print(text)
print('-' * 40)
print(' '.join(tokens))
print('-' * 40)
print(' '.join(purged_tokens))

А как он, с точки зрения банальной физики, может не лагать при клике на что-либо новое? Так или иначе нужно отправлять запрос, ждать ответ, парсить новый жс… А если prefetching — то в чем преимущество перед тем, чтобы загрузить все одним блоком и забыть?
----------------------------------------
а как он с точка зрение банальный физика мочь не лагать при клик на что либо новый так или иначе нужно отправлять запрос ждать ответ парсить новый жс а если prefetching то в чем преимущество перед тем чтобы загрузить всё один блок и забыть
----------------------------------------
точка зрение банальный физика лагать клик либо иначе отправлять запрос ответ парсить жс prefetching преимущество загрузить блок забыть
